In [11]:
from simpletransformers.classification import ClassificationModel
import pandas as pd
import sklearn
import logging
from pathlib import Path 

from sklearn.model_selection import train_test_split

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

In [4]:
def get_train_args(model_name):
    train_args = {
        "output_dir": "reproduction_exp2/" + model_name,
        "cache_dir": "cache/",
        "best_model_dir": "reproduction_exp2/" + model_name + "/best_model/",

        "fp16": False,
        "fp16_opt_level": "O1",
        "max_seq_length": 128,
        "train_batch_size": 16,
        "eval_batch_size": 16,
        "gradient_accumulation_steps": 1,
        "num_train_epochs": 5,
        "weight_decay": 0,
        "learning_rate": 5e-6,
        "adam_epsilon": 1e-9,
        "warmup_ratio": 0.06,
        "warmup_steps": 0,
        "max_grad_norm": 1.0,
        "do_lower_case": False,

        "logging_steps": 50,
        "evaluate_during_training": True,
        "evaluate_during_training_steps": 0,
        "evaluate_during_training_verbose": True,
        "use_cached_eval_features": False,
        "save_eval_checkpoints": False,
        "save_steps": 0,
        "no_cache": True,
        "save_model_every_epoch": True,
        "tensorboard_dir": None,

        "overwrite_output_dir": True,
        "reprocess_input_data": True,

        "n_gpu": 1,
        "silent": False,
        "use_multiprocessing": True,

        "wandb_project": None,
        "wandb_kwargs": {},

        "use_early_stopping": False,
        "early_stopping_patience": 4,
        "early_stopping_delta": 0,
        "early_stopping_metric": "f1",
        "early_stopping_metric_minimize": False,

        "manual_seed": 9721,
        "encoding": None,
        "config": {},
    }
    return train_args

def precision_macro(y_true, y_pred):
    return sklearn.metrics.precision_score(y_true, y_pred, average='macro')
def recall_macro(y_true, y_pred):
    return sklearn.metrics.recall_score(y_true, y_pred, average='macro')
def f1_macro(y_true, y_pred):
    return sklearn.metrics.f1_score(y_true, y_pred, average='macro')

def reformat_df(data_df):
    df = data_df[['argument1', 'argument2', 'is_same_side']].copy()
    df["is_same_side"] = df["is_same_side"].astype(int)
    df = df.rename(columns={"argument1": "text_a", "argument2": "text_b", "is_same_side" : 'labels'})
    return df

In [8]:
data_dir = Path("data/pubdata")
# read datasets and reformat each datapoint to [arg1, arg2, label] format
datasets = {
    "random" : (reformat_df(pd.read_json(data_dir / "exp2_random_train.jsonl", lines=True)), reformat_df(pd.read_json(data_dir / "exp2_random_test.jsonl", lines=True))),
    "disjoint-within" : (reformat_df(pd.read_json(data_dir / "exp2_disjoint-within_train.jsonl", lines=True)), reformat_df(pd.read_json(data_dir / "exp2_disjoint-within_test.jsonl", lines=True))),
    "disjoint-cross" : (reformat_df(pd.read_json(data_dir / "exp2_disjoint-cross_train.jsonl", lines=True)), reformat_df(pd.read_json(data_dir / "exp2_disjoint-cross_test.jsonl", lines=True))),
    "single" : (reformat_df(pd.read_json(data_dir / "exp2_single_train.jsonl", lines=True)), reformat_df(pd.read_json(data_dir / "exp2_single_test.jsonl", lines=True))),
}

In [9]:
datasets.keys()

dict_keys(['random', 'disjoint-within', 'disjoint-cross', 'single'])

In [12]:
# run experiments

model_to_test = ('albert', 'albert-base-v2')

for exp_name in datasets.keys():
    
    (train_df, test_df) = datasets[exp_name]
    
    # get training args
    train_args = get_train_args("exp_" + exp_name)
    
    print("******************************************")
    print(exp_name)
    print("******************************************")

    # split a dev set from the training data
    train_df, dev_df = train_test_split(train_df, test_size = 0.1, random_state = 9721)
    print(len(train_df))
    print(len(dev_df))
    print(len(test_df))

    # Create a ClassificationModel
    model = ClassificationModel(model_to_test[0], model_to_test[1], num_labels=2, use_cuda=True, cuda_device=0, args=train_args)

    # Train the model
    model.train_model(train_df, eval_df=dev_df, precision=precision_macro, recall=recall_macro, f1=f1_macro)

    # Eval best model
    model = ClassificationModel(model_to_test[0], train_args['best_model_dir'], num_labels=2, use_cuda=True, cuda_device=0, args=train_args)
    result, model_outputs, wrong_predictions = model.eval_model(
        test_df, acc=sklearn.metrics.accuracy_score, precision=precision_macro, recall=recall_macro, f1=f1_macro
    )
    
    print(result)

******************************************
random
******************************************
76304
8479
9421


Some weights of the model checkpoint at albert-base-v2 were not used when initializing AlbertForSequenceClassification: ['predictions.dense.weight', 'predictions.dense.bias', 'predictions.decoder.weight', 'predictions.LayerNorm.weight', 'predictions.decoder.bias', 'predictions.LayerNorm.bias', 'predictions.bias']
- This IS expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing AlbertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of AlbertForSequenceClassification were not initialized from the model checkpoint at albert-base-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You sho

  0%|          | 0/76304 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/4769 [00:00<?, ?it/s]

...

INFO:simpletransformers.classification.classification_model:{'mcc': 0.7732191748588845, 'tp': 4076, 'tn': 3273, 'fp': 387, 'fn': 546, 'auroc': 0.9681846798277661, 'auprc': 0.9766034523040632, 'precision': 0.885158831477951, 'recall': 0.8880658078611914, 'f1': 0.8862769745380628, 'eval_loss': 0.35991413664889904}
INFO:simpletransformers.classification.classification_model: Training of albert model complete. Saved to reproduction_exp2/exp_single.
INFO:simpletransformers.classification.classification_utils: Converting to features started. Cache is not used.


  0%|          | 0/11391 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/712 [00:00<?, ?it/s]

INFO:simpletransformers.classification.classification_model:{'mcc': 0.32557828784723963, 'tp': 2866, 'tn': 4772, 'fp': 1590, 'fn': 2163, 'auroc': 0.7299838084660681, 'auprc': 0.6933849794123881, 'acc': 0.6705293652883856, 'precision': 0.6656407795391679, 'recall': 0.6599866014462862, 'f1': 0.66103924972038, 'eval_loss': 0.7946493366626588}


{'mcc': 0.32557828784723963, 'tp': 2866, 'tn': 4772, 'fp': 1590, 'fn': 2163, 'auroc': 0.7299838084660681, 'auprc': 0.6933849794123881, 'acc': 0.6705293652883856, 'precision': 0.6656407795391679, 'recall': 0.6599866014462862, 'f1': 0.66103924972038, 'eval_loss': 0.7946493366626588}
